# Data Anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import os  
import sys
import plotly.io as pio
pio.renderers.default = "notebook_connected"

sys.path.append("../../../")
sys.path.append('../../python')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage30yr""","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage30yr""","""Unknown""",669,"""Laurianneshire""","""AutoLoans""","""Accepted"""
"""MoneyMarketSavingsAccount""","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmieshire""","""DepositAccounts""","""Accepted"""
"""UPlusFinPersonal""","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccounts""","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

filename,PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_PREDICTOR_0,Decision_Outcome
str,str,f64,str,str,str,bool
"""../../../../data/SampleHDS.jso…","""922743504929640761""",1.7418e19,"""3297677166849977077""","""5188124373359796199""","""9095368589361968165""",false
"""../../../../data/SampleHDS.jso…","""5055478732360118593""",1.4458e19,"""9151086011386372298""","""5188124373359796199""","""8153881765308184972""",true
"""../../../../data/SampleHDS.jso…","""7450752871781526164""",1.3309e19,"""8076650117006862747""","""8067694184923790887""","""9095368589361968165""",false
"""../../../../data/SampleHDS.jso…","""5055478732360118593""",6.7303e18,"""6147649868770868734""","""12509399971611518094""","""9095368589361968165""",false
"""../../../../data/SampleHDS.jso…","""922743504929640761""",1.8388e19,"""15203907127025817554""","""12509399971611518094""","""13748319429950525116""",true
"""../../../../data/SampleHDS.jso…","""7450752871781526164""",1.6360e19,"""3812447758722794602""","""117525414105497968""",null,false
"""../../../../data/SampleHDS.jso…","""7450752871781526164""",6.4271e18,"""16968748742397683417""","""12509399971611518094""","""13748319429950525116""",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'filename': 'filename',
 'Customer_MaritalStatus': 'PREDICTOR_1',
 'Customer_CLV': 'PREDICTOR_2',
 'Customer_City': 'PREDICTOR_3',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder="../../../../data/",
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


filename,PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,f64,str,str,str,str
"""../../../../data/SampleHDS.jso…","""4805560467625538498""",8.8030e18,"""16603811012619439722""","""FirstMortgage30yr""","""11793098893927356311""","""Rejected"""
"""../../../../data/SampleHDS.jso…","""16202350137977306073""",1.1945e19,"""2204981920139629968""","""FirstMortgage30yr""","""16730447032076587570""","""Accepted"""
"""../../../../data/SampleHDS.jso…","""843728177941714718""",1.3050e18,"""7886312263181228367""","""MoneyMarketSavingsAccount""","""11793098893927356311""","""Rejected"""
"""../../../../data/SampleHDS.jso…","""16202350137977306073""",2.7500e18,"""9412613614002633636""","""BasicChecking""","""11793098893927356311""","""Rejected"""
"""../../../../data/SampleHDS.jso…","""4805560467625538498""",1.2088e19,"""5873390260891191394""","""BasicChecking""","""15451477672086794446""","""Accepted"""
"""../../../../data/SampleHDS.jso…","""843728177941714718""",3.3874e18,"""17233004760106334083""","""UPlusFinPersonal""",null,"""Rejected"""
"""../../../../data/SampleHDS.jso…","""843728177941714718""",2.8641e18,"""8063772434521972735""","""BasicChecking""","""15451477672086794446""","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

filename,PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,f64,str,str,str,str
"""../../../../data/SampleHDS.jso…","""3975837263940628186""",7.1609e18,"""10014847475767181371""","""FirstMortgage30yr""","""2796501546597985419""","""Rejected"""
"""../../../../data/SampleHDS.jso…","""5825735455856901651""",7.1721e18,"""6325508596068253384""","""FirstMortgage30yr""","""11250185154281592345""","""Accepted"""
"""../../../../data/SampleHDS.jso…","""12497433115501929529""",1.5166e19,"""8962923969799790943""","""MoneyMarketSavingsAccount""","""2796501546597985419""","""Rejected"""
"""../../../../data/SampleHDS.jso…","""5825735455856901651""",3.6952e18,"""11232057115290894263""","""BasicChecking""","""2796501546597985419""","""Rejected"""
"""../../../../data/SampleHDS.jso…","""3975837263940628186""",7.9703e18,"""9910654448942286122""","""BasicChecking""","""9104528945110680687""","""Accepted"""
"""../../../../data/SampleHDS.jso…","""12497433115501929529""",7.2435e18,"""1692243460214910783""","""UPlusFinPersonal""",null,"""Rejected"""
"""../../../../data/SampleHDS.jso…","""12497433115501929529""",1.7027e19,"""15461126931171093670""","""BasicChecking""","""9104528945110680687""","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

filename=filename
Customer_MaritalStatus=PREDICTOR_1
Customer_CLV=PREDICTOR_2
Customer_City=PREDICTOR_3
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str
"""98383438131944770""",1.6117e19,"""10724497247547814062""","""FirstMortgage30yr""","""17401590597689748271""","""Rejected"""
"""18213068371331517484""",7.5518e18,"""13234174383800351398""","""FirstMortgage30yr""","""11444648475706298877""","""Accepted"""
"""1145919506863053315""",4.9612e18,"""1541302271169447543""","""MoneyMarketSavingsAccount""","""17401590597689748271""","""Rejected"""
"""18213068371331517484""",1.0288e19,"""499527667868569097""","""BasicChecking""","""17401590597689748271""","""Rejected"""
"""98383438131944770""",2.3613e18,"""6702873829387909661""","""BasicChecking""","""1430272949244540913""","""Accepted"""
"""1145919506863053315""",1.7669e19,"""15497387757201345340""","""UPlusFinPersonal""",null,"""Rejected"""
"""1145919506863053315""",1.2194e19,"""7432436853168423243""","""BasicChecking""","""1430272949244540913""","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.



ComputeError: AttributeError: 'int' object has no attribute 'encode'